## Background: understanding the math in ML papers

The most popular introductions to diffusion models derive them from starting points like non-equilibrium thermodynamics, or make assumptions that you're an expert in the generative modeling literature, probability theory, [Langevin dynamics](https://en.wikipedia.org/wiki/Metropolis-adjusted_Langevin_algorithm), and so on. I don't have anything against these articles. In fact, I think they're excellent, they help me develop my understanding of diffusion models, and they're very well written. But there isn't really an article for people without these backgrounds.

There isn't an introduction to diffusion models for coders. This is that introduction!

I would go so far as to say that the math-first approach can hinder further development of the ideas. I think it hamstrings our thinking into forms that fit math (e.g. Gaussian distributions and so on) whereas history has shown that progress in deep learning is often made through leaps that aren't informed by math.

There are two main ways to derive diffusion models.

1. The first is to assume that you generate images by iteratively adding Gaussian noise. Then try to estimate the reverse process using variational inference (?).
2. The second is to try to sample from the data distribution using the gradient of the log likelihood, so-called "score-based" models.

(I think Jeremy is developing another "hacker" approach to derive it as a natural consequence of GANs.)

## Deep Unsupervised Learning using Nonequilibrium Thermodynamics

_Jascha Sohl-Dickstein, Eric A. Weiss, Niru Maheswaranathan, and Surya Ganguli. 2015, June._

https://github.com/Sohl-Dickstein/Diffusion-Probabilistic-Models

https://arxiv.org/abs/1503.03585

In [ ]:
from fastai.vision.all import *

### 2.1. Forward Trajectory

::: {.pt-3 .pb-1 .px-3 .my-3 .border .rounded .shadow-sm}

We label the [data distribution $q(\mathbf{x}^{(0)})$]{.hl-indigo}. The data distribution is gradually converted into a [well behaved (analytically tractable) distribution $\pi(\mathbf y)$]{.hl-green} by repeated application of a [Markov diffusion kernel $T_\pi(\mathbf y|\mathbf y'; \beta)$]{.hl-yellow} for $\pi(\mathbf y)$, where [$\beta$ is the diffusion rate]{.hl-blue},

\begin{align}
\pi(\mathbf y) &= \int d\mathbf y' T_\pi (\mathbf y|\mathbf y';\beta) \pi(\mathbf y') \tag{1} \\
q(\mathbf x^{(t)}|\mathbf x^{(t-1)}) &= T_\pi(\mathbf x^{(t)} | \mathbf x^{(t-1)}; \beta_t) \tag{2}
\end{align}

:::

There are four objects introduced here, highlighted in four different colors. The two equations describe the relationships between these objects.

#### Random variables vs probability density functions

The first concept required to understand this is that of representing random variables (vectors in this case) in terms of their _probability density functions_. Mathematicians do this to enable the analysis of random variables. Although it isn't explicitly mentioned here, $\pi$, $q$, and $T_\pi$ are all probability density functions. The pdf $\pi(\mathbf y)$ is a function that accepts a vector as input and returns the probability that the provided value of $\mathbf y$ might occur.

Pdfs have been studied for a very long time, so they have many properties which are now well understood by mathematicians.

The authors of this paper use the notation that the input symbol of the pdf is the name of the random vector described by said pdf. For example, $\mathbf y$ is a _random vector_ whose behavior is completely described by the probability density function $\pi$.

Coders often find it easier to think in terms of the random variables and vectors, however, since they're closer to what gets written into our programs in the end. Let's shift focus to the random vectors introduced here.

#### Data distribution $q(\mathbf x^{(0)})$

There are a few more subtleties in the notation used for these random vectors. The first random vector we were introduced to is $\mathbf x^{(0)}$. $x$ is often used for the object that's ultimately being modeled. In this case, it's a random vector representing the _data distribution_. For example, if we were using the MNIST dataset, then $\mathbf x^{(0)}$ is a randomly sampled image that looks like an MNIST digit. It may not be an actual image from the dataset -- this is the goal of the abstraction "data distribution", it assumes some underlying data generating system, of which the actual MNIST dataset is only one sample.

The superscript $(0)$ here does not mean "to the power of". This is suggested by the brackets surrounding the zero, a notation commonly used in some domains including physics. The use of the superscript already suggests that we might see the sequence continue, for example, $\mathbf x^{(1)}$, $\mathbf x^{(2)}$, and so on.

#### Analytically tractable distribution $\pi(\mathbf y)$

The use of $\mathbf y$ in $\pi(\mathbf y)$ suggests that the authors are trying to highlight a more general property about the functions $\pi$ and $T_\pi$ that's totally unrelated to the data $\mathbf x$.

What does "analytically tractable" distribution mean? I'm not 100% sure but I think it means a relatively straightforward formula that we can type out in Python when defining a simple function. In other words, the authors claim that we can gradually convert the data distribution (some hypothetical Python function that returns novel images that look like MNIST digits) into a relatively straightforward mathematical formula that can be programmed in Python!

If we peak ahead to Table App. 1[^peaking], we can see some examples of these objects. However, instead of $\pi(\mathbf y)$ we now see $\pi(\mathbf x^{(T)})$, which may be set to $\mathcal N (\mathbf x^{(T)}; \mathbf 0, \mathbf I)$ (Gaussian) or $\mathcal B(\mathbf x^{(T)}; 0.5)$ (Binomial).

This is a new notation, let's describe it. $\mathcal N$ and $\mathcal B$ refer to pdfs of two well-known distributions: Gaussian and Binomial. The semicolon separates function inputs on the left from parameters on the right. It's not immediately obvious what the difference is between an input and parameter of a function, they only really make sense for classes of functions aka models. For example, the equation of a Gaussian pdf can be written down as a function of the inputs as well as two parameters. Recall from earlier that this is why we might call it an analytically tractable distribution.

[^peaking]: It's often a good idea to peak ahead while reading a scientific paper. In fact, I often find it most useful to jump back and forth many times and very rarely read papers linearly. I was introduced to this and other ideas in the wonderful paper: ["How to Read a Paper"](https://web.stanford.edu/class/ee384m/Handouts/HowtoReadPaper.pdf) by Srinivasan Keshav.

#### Equation 1

I'm not sure that my interpretation of this equation is correct. Please let me know of any mistakes.

$$
\pi(\mathbf y) = \int d\mathbf y' T_\pi (\mathbf y|\mathbf y';\beta) \pi(\mathbf y') \tag{1}
$$

## Old

You can think of a _distribution_ as a Python function with a random output. A _data_ distribution is such a random function that returns some example from a given dataset. In _probability theory_, we study the behavior random objects and randomness in general, including these sorts of random functions.

$\mathbf{x}^{(0)}$ represents the image, and $q$ represents the "distribution" of images -- which we said we would think of as a function that randomly returns one of the images in our dataset.

You might be wondering why $\mathbf{x}^{(0)}$ is an input to $q$ if it gets returned from our function. That's because in probability theory, we define the distribution as a function that accepts an image (in this case) and returns a number between 0 and 1 that tells us how likely it is that that image would occur in our dataset. This definition turns out to be very useful to study these sorts of random variables. Although it isn't immediately obvious what the likelihood of any given MNIST image is. But we can quite easily sample from the MNIST "distribution" without knowing that!

For example, here's how we might construct the MNIST dataset:

In [ ]:
path = untar_data(URLs.MNIST)
paths = list(walk(path))
p = random.choice(paths)
im = Image.open(p)
im

Every time you run the cell above, you'll "sample" from the MNIST dataset, in other words, you'll sample an image $\mathbf{x}^{(0)}$ from the distribution $q(\mathbf{x}^{(0)})$.

Although this isn't quite right! The hypothesis in the paper, and in generative modeling in general, is that this dataset itself is only a sample from some underlying data distribution. We hypothesis that there exists some random function that would return other images that look a lot like MNIST images, but aren't exactly the same -- the underlying data distribution $q$.

The goal of generative modeling is to learn how to sample from such a data distribution, starting out from a sample dataset drawn from that distribution. This is a useful mathematical framing, although it might be simpler to say that the goal is to learn how to generate novel images that look similar to images from a given dataset.

The tools of probability theory allow us to better understand the problem, and devise potential solutions, by framing it in this slightly weird format.

We haven't gotten to what the $(0)$ above the $\mathbf{x}$ means yet.

> The data distribution is gradually converted into a well behaved (analytically tractable) distribution $\pi(y)$ by repeated application of a Markov diffusion kernel $T_\pi(y|y′; \beta)$ for $\pi(y)$, where $\beta$ is the diffusion rate,

Okay, there's _tons_ to unpack in this sentence!

> The data distribution is gradually converted into a well behaved (analytically tractable) distribution $\pi(y)$ ...

This is starting to explaining why there's a $(0)$ above the $\mathbf{x}$ -- because we'll be gradually converting the data distribution (think MNIST images) into something else.

We'll be converting it into a "well behaved (analytically tractable) distribution". What does that mean?

I'm not too sure what well behaved is supposed to mean here to be honest! However, I think "analytically tractable" means a relatively straightforward formula that we can type out in Python when defining a simple function. In other words, the authors claim that we can gradually convert the data distribution (some hypothetical Python function that returns novel images that look like MNIST digits) into a relatively straightforward mathematical formula that can be programmed in Python! If we can do that, we've solved the problem we're trying to solve -- of generative modeling.

I don't think this is true just yet. This is only the forward process!

They call that final distribution $\pi(\mathbf y)$

Where do $\pi$ and $\mathbf y$ come from?

I'm not sure! But I think the intention behind using $\mathbf y$ instead of something like $\mathbf x$ is to signify that they're different random variables.

A "Markov diffusion kernel" basically means a function that is iteratively applied to some input.

### 2.1 Forward trajectory

We label the data distribution $q(\mathbf x^{(0)})$.

\begin{equation}
\pi(\mathbf y) = \int d\mathbf y' T_\pi (\mathbf y|\mathbf y';\beta) \pi(\mathbf y')
\tag{1}
\end{equation}

::: {.callout-note}

### What does this mean?

I think the point of this equation is to say that $T_\pi$ is some "kernel" which if convolved with the distribution $\pi$ gives another distribution of the same functional form. The point of the equation is to tell us how $\pi$ and $T_\pi$ relate to each other. I'm not sure why this property is important though... and whether/why it's satisfied by Gaussian and Binomial distributions.

I think the point is that applying $T_\pi$ to $\pi$ has no effect! Remember that this is about distributions, so the distribution is the same after "applying" the kernel.

Let's reframe this in terms of the forward process:

$$
q(\mathbf x^{(t)}) = \int d\mathbf x^{(t-1)} q(\mathbf x^{(t)}|\mathbf x^{(t-1)}) q(\mathbf x^{(t-1)})
$$

(By the way, this is equivalent to saying the following, due to the law of total probability -- and some other assumptions?)

\begin{equation}
q(\mathbf x^{(0\cdots T)}) = q(\mathbf x^{(0)}) \prod_{t=1}^T q(\mathbf x^{(t)}|\mathbf x^{(t-1)})
\tag{3}
\end{equation}

For a Gaussian kernel:

$$
q(\mathbf x^{(t)}) = \int d\mathbf x^{(t-1)} \mathcal N(\mathbf x^{(t)}; \mathbf x^{(t-1)}\sqrt{1-\beta_t}, \mathbf I \beta_t) q(\mathbf x^{(t-1)})
$$

Since its a Gaussian:

$$
q(\mathbf x^{(t)}) = \int d\mathbf x^{(t-1)} \mathcal N(\mathbf x^{(t)} - \mathbf x^{(t-1)}\sqrt{1-\beta_t}; \mathbf 0, \mathbf I \beta_t) q(\mathbf x^{(t-1)})
$$

Which is the definition of convolution of two functions:

$$
q(\mathbf x^{(t)}) = \mathcal N(\mathbf x^{(t)}; \mathbf 0, \mathbf I \beta_t) \ast q(\mathbf x^{(t-1)})
$$

Which, for independent distributions, is equivalent to adding their random variables:

$$
\mathbf x^{(t)} = \sqrt{\beta_t} \epsilon_t + q(\mathbf x^{(t-1)})
$$

where

$$
\epsilon_t = \mathcal N(\mathbf x^{(t)}; \mathbf 0, \mathbf I)
$$

The equation in terms of probability density functions is useful to work with mathematically, whereas the equation in terms of random variables is much closer to what the final code looks like.

:::

Forward diffusion kernel:

\begin{equation}
q(\mathbf x^{(t)}|\mathbf x^{(t-1)}) = T_\pi(\mathbf x^{(t)} | \mathbf x^{(t-1)}; \beta_t)
\tag{2}
\end{equation}

Forward diffusion process:

\begin{equation}
q(\mathbf x^{(0\cdots T)}) = q(\mathbf x^{(0)}) \prod_{t=1}^T q(\mathbf x^{(t)}|\mathbf x^{(t-1)})
\tag{3}
\end{equation}

### 2.2 Reverse trajectory

\begin{equation}
p(\mathbf x^{(T)}) = \pi(\mathbf x^{(T)})
\tag{4}
\end{equation}

\begin{equation}
p(\mathbf x^{(0\cdots T)}) = p(\mathbf x^{(T)}) \prod_{t=1}^T p(\mathbf x^{(t-1)}|\mathbf x^{(t)})
\tag{5}
\end{equation}

### Gaussian distribution

Well-behaved (analytically tractable) distribution:

$$\pi(\mathbf x^{(T)}) = \mathcal N (\mathbf x^{(T)}; \mathbf 0, \mathbf I)$$

Forward diffusion kernel:

\begin{equation}
q(\mathbf x^{(t)}|\mathbf x^{(t-1)}) = \mathcal N(\mathbf x^{(t)}; \mathbf x^{(t-1)}\sqrt{1-\beta_t}, \mathbf I \beta_t)
\end{equation}

Reverse diffusion kernel:

$$p(\mathbf x^{(t-1)}|\mathbf x^{(t)}) = \mathcal N(\mathbf x^{(t-1)}; \mathbf f_\mu(x^{(t)}, t), \mathbf f_\Sigma(x^{(t)}, t))$$

Training targets:

$$\mathbf f_\mu(x^{(t)}, t), \mathbf f_\Sigma(x^{(t)}, t)), \beta_{1\cdots T}$$

Forward distribution:

\begin{equation}
q(\mathbf x^{(0\cdots T)}) = q(\mathbf x^{(0)}) \prod_{t=1}^T q(\mathbf x^{(t)}|\mathbf x^{(t-1)})
\end{equation}

Reverse distribution:

\begin{equation}
p(\mathbf x^{(0\cdots T)}) = \pi(\mathbf x^{(T)}) \prod_{t=1}^T p(\mathbf x^{(t-1)}|\mathbf x^{(t)})
\end{equation}

## Rough

I think the first and second mean:

$$\pi(\mathbf x^{(T)}) = \mathcal N (\mathbf x^{(T)}; \mathbf 0, \mathbf I) = \int d\mathbf ? T_\pi (\mathbf x^{(T)}|\mathbf ?;\beta) \pi(\mathbf ?)$$

$$\pi(\mathbf x^{(T)}) = \mathcal N (\mathbf x^{(T)}; \mathbf 0, \mathbf I) = \int d\mathbf x \mathcal N(\mathbf x^{(T)}; \mathbf x \sqrt{1-\beta_t}, \mathbf I \beta_t) \pi(\mathbf x)$$

I have _no idea_ what the integral means!

## Discovering diffusion

We define a "forward" process, with known parameters, that starts from our data distribution (actual images) and applies a series of very tiny corruptions until we eventually reach pure noise.

If we do this within some constraints (Markov process, Gaussian transitions, Gaussian noise at the end, small step sizes), then the reverse process will have the same functional form (Markov process, Gaussian transitions, Gaussian noise at the beginning, small step sizes)! The only part that we don't yet know is the parameters of this reverse process.

The go-to method of estimating parameters of a probabilistic model is to find the parameters that maximize the log likelihood function. While we can do that with closed-form solutions that we can write down for simple distributions, it becomes impossible for more complex distributions. And it can't be computed because it would require integrating over a high-dimensional and continuous space. Instead, there's a trick to optimize a lower bound of it, called the "ELBO", which in practice is just as good but quite a lot easier to calculate.

The original paper worked through the math to figure out what the ELBO is if we estimate the mean and variance of the reverse diffusion kernel.

DDPM improved on that by 

Start by assuming that your data was generated by a Markov process starting from pure random noise (this has really convenient mathematical properties) and gradually refining it.

If the refinement is tiny at each step, then there's some math that says that the forward and reverse process have the same functional form. Meaning that we should be able to 